# xml文件详解
- 以odoo为根标签
        <odoo>
            <data noupdate="1">
                <!-- Only loaded when installing the module (odoo-bin -i module) -->
                <operation/>
            </data>

            <!-- (Re)Loaded at install and update (odoo-bin -i/-u) -->
            <operation/>
        </odoo>

# 核心的operation
## record
- 他有以下属性
    - model:需要创建或更新的模型
    - id：记录的id，强烈推荐定义它
    - context：创建记录时需要使用的上下文
    - forcecreate：在更新模式下如果没有该记录是否要强制创建。默认为True，要求有id
- 示例：
    record model="ir.cron" id="cron_clear_session_store"

## field
- 每个record都是由多个field组成的。这些field定义了创建record时要设置的值。没有field的记录将使用所有默认值(创建)或不执行任何操作(更新)。
- 他有以下属性：
    - name：field有一个强制的name属性，即要设置的字段名，以及定义值本身的各种方法
    - 如果没有为该field提供值，则在该字段上隐式设置False。可用于清除字段，或避免使用该字段的默认值。
    - search：如果该field是一个关联字段，那么search就是一个domain，如果是Many2one将搜索得到的第一个结果作为该字段的值
    - ref：它的值是一个有效的外部id。大多数是many2one和引用字段
    - type：如果提供，则表示来转换该字段的内容。字段的内容可以通过使用file属性的外部文件提供，也可以通过节点的主体提供。type类型有：
        - xml，html：
            - 可以将field子元素提取为一个独立的文档，同时可以使用%external_id的方式调用其他的视图作为独立的文档。（如果想输出%,使用%%)
                    <record id="view_calendar_event_calendar" model="ir.ui.view">
                    <field name="name">calendar.event.calendar</field>
                    <field name="model">calendar.event</field>
                    <field name="priority" eval="2"/>
                    <field name="arch" type="xml">
                        <calendar string="Meetings" date_start="start" date_stop="stop" date_delay="duration" all_day="allday"
                            readonly_form_view_id="%(calendar.view_calendar_event_form_popup)s"
                            event_open_popup="true"
                            event_limit="3"
                            color="partner_id">
                            <field name="name"/>
                            <field name="partner_ids" write_model="calendar.contacts" write_field="partner_id" avatar_field="image_small"/>
                            <field name="is_highlighted" invisible="1"/>
                        </calendar>
                    </field>
                </record>
        - file:
            - 确保字段内容是当前模型中有效的文件路径，将 (module,path) 作为字段值保存
        - char:
            - 直接将字段内容设置为字段的值而不作更改
        - base64:
            - base64对字段的内容进行编码，与文件属性结合使用，可以将图像数据加载到附件中
        - int:将字段内容转换为整型并将其设置为字段的值
        - float
        - list/tuple:应该包含任意数量的值元素，具有与字段相同的属性，每个元素解析为生成的元组或列表中的一个项，生成的集合设置为字段的值.(没懂)
    - eval:eval属性只是计算提供给它的任何Python表达式，并将结果设置为字段的值。
        - 计算上下文包含各种模块(time, datetime, timedelta, relativedelta)，一个用于解析外部标识符(ref)的函数，以及当前字段的模型对象(如果适用的话)
        
## delete
- 它可以移除之前已经定义好的任意数量的record
- 他有以下属性
    - model，id，search
    - 示例：delete model="account.fiscal.position.template" search="[('chart_template_id','=',ref('l10n_fr_pcg_chart_template'))]"/
    
## function
- function标签使用提供的参数调用模型上的方法。它有两个强制参数model和name，分别指定要调用的模型和方法的名称。
- 参数的提供：
    - 参数可以使用eval(应该计算为调用该方法的一系列参数)或value元素(参见列表值)提供。
            <function model="account.chart.template" name="try_loading_for_current_company">
                <value eval="[ref('l10n_at.austria_chart_template')]"/>
            </function>
            
            <data noupdate="1">
                <record name="partner_1" model="res.partner">
                    <field name="name">Odude</field>
                </record>

                <function model="res.partner" name="send_inscription_notice"
                    eval="[[ref('partner_1'), ref('partner_2')]]"/>

                <function model="res.users" name="send_vip_inscription_notice">
                    <function eval="[[('vip','=',True)]]" model="res.partner" name="search"/>
                </function>
            </data>

# xml的一些简写标签
## menuitem
- 是用来定义ir.ui.menu记录，即菜单
- 属性：
    - parent：父级菜单的id，如果没有定义它，则根据它的名字以/进行层级划分，然后根据这个划分自动创建中间菜单；或者它成为一个“顶级菜单"（但是并不是真正的顶级)
    - name:名字，如果没有定义，从action获取，如果还没有就从id获取
    - groups：定义权限组，如果前缀是-，则表示剔除该权限组
    - id

## template
- 只是创建给arch的视图
- 可选属性：
    - id：
    - name, inherit_id, priority
    - primary：如果和inherit共用，则是继承
    - groups：
    - page：如果设置为True，则该模板是一个网站页面(可链接，可删除)
    - optional：enabled或者disabled，视图是否启用或禁用。默认启用